<a href="https://colab.research.google.com/github/anndawn/COVID_procurement-documentation/blob/main/dashboard_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
%cd gdrive/MyDrive/dashboard/

/content/gdrive/MyDrive/dashboard


In [ ]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
url = 'https://app.powerbi.com/view?r=eyJrIjoiOTMwNjRjZmYtNWM3YS00NGFlLTllNDItMDlhYWUxZWRiY2Y5IiwidCI6IjFhNDUzNDhmLTAyYjQtNGY5YS1hN2E4LTc3ODZmNmRkMzI0NSIsImMiOjh9&pageName=ReportSection'
driver.get(url)

In [ ]:
# next page arrow
elem = driver.find_element_by_xpath('//a[@class="middleText"]/following-sibling::a')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [ ]:
# flip four pages
for i in range (0,4):
  elem.click()

In [ ]:
# right click on the table
# select show table
action = ActionChains(driver)
elementLocator = driver.find_element_by_xpath('//div[@class="bodyCells"]');
action.context_click(on_element = elementLocator).perform()
driver.find_element_by_xpath('//div[@class="label"]').send_keys(Keys.ENTER)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [ ]:
# ele_child is the down arrow on the top table
elem_long=driver.find_element_by_xpath("//*[@class='pivotTableContainer']")
ele_child=elem_long.find_element_by_xpath('//*[@class="scroll-bar-div"][2]').find_elements_by_xpath('.//div[2]')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [ ]:
# arr_right is the right arrow of the top table
arr_right=elem_long.find_element_by_xpath('//*[@class="scroll-bar-div"][1]').find_elements_by_xpath('.//div[2]')

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [ ]:
# define click event after read the records of the current screen
def click_event():
    for i in range (0,60):
        ele_child[0].click();
        time.sleep(0.02);
        if i>18:
            result=driver.find_element_by_xpath("//*[@class='bodyCells']//div[1]")
            children=result.find_elements_by_xpath('./*')
            if len(children)==2:
                break

In [ ]:
# read data from the current screen table
def get_values():
    cols=["Institution_name","Item_desp","Supplier_name","Service_desp","Unit_price","Quantity","UOM","Date_order","Order_amount","Date_payments","Payments"]
    df=pd.DataFrame(columns=cols)
    for i in range (0,10):
        path="//*[@class='bodyCells']//div[1]//div[1]"
        column=driver.find_element_by_xpath(path)
        col_child= column.find_elements_by_xpath('./*')[i].find_elements_by_xpath(".//div[@class='pivotTableCellWrap cell-interactive ']|div[@class='pivotTableCellWrap cell-interactive tablixAlignRight ']")
        col_values = [child.get_attribute('title') for child in col_child]
        df[cols[i]]=pd.Series(col_values)
    payment_column=driver.find_elements_by_xpath("//*[@class='bodyCells']//div[1]//div[2]//div[1]//child::div[@class='pivotTableCellWrap cell-interactive tablixAlignRight ']")
    value_payment= [child.get_attribute('title') for child in payment_column]
    df["Payments"]=pd.Series(value_payment)
    return df

In [ ]:
# right click until payment column shows
for i in range(80):
    arr_right[0].click()

In [ ]:
l=driver.find_element_by_xpath("//div[@title='Date_of_Order']")
l.click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


In [ ]:
l2=driver.find_element_by_xpath("//div[@title='Date_of_Order']")
l2.click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


In [ ]:
# test getting value of the current screen
df0=get_values()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# test writing into google drive
# df0.to_csv("data_001.csv")

In [ ]:
df0

,Institution_name,Item_desp,Supplier_name,Service_desp,Unit_price,Quantity,UOM,Date_order,Order_amount,Date_payments,Payments
0,Western Cape - Health,Food parcels,MFJ FOODS,0,10.49,470,0,31/12/2020,"R4,930.30",02/12/2021,"R4,930"
1,Western Cape - Health,Food parcels,MFJ FOODS,0,280.10,64,0,31/12/2020,"R17,926.40",02/12/2021,"R17,926"
2,KZN DEPARTMENT OF SOCIAL DEVELOPMENT,Other,Sinendaba 94 Projects Solutions,catering for Homeless people,"3,720.00",120,Each,31/12/2020,"R446,400.00",02/09/2021,"R446,400"
3,KZN DEPARTMENT OF SOCIAL DEVELOPMENT,Other,LUTAZO TRADING ENTERPRISE,catering for Homeless people,"3,720.00",13,Each,31/12/2020,"R48,360.00",02/09/2021,"R48,360"
4,Western Cape - Health,Other,New Horizon Metals,medical Service,"1,781.58",6,0,31/12/2020,"R10,689.48",03/05/2021,"R10,689"
5,Western Cape - Health,Other,ILEX (SOUTH AFRICA),Stationery,"12,898.40",2,KG,31/12/2020,"R25,796.80",14/01/2021,"R25,797"
6,Western Cape - Health,Other,RESPIRATORY CARE AFRICA,Transport Service,"57,473.55",1,BX,31/12/2020,"R57,473.55",14/01/2021,"R57,474"
7,Western Cape - Health,PPE000 Other PPEs,COMPASS MEDICAL WASTE SERVICES,0,443.81,32,0,31/12/2020,"R14,201.92",17/02/2021,"R14,202"
8,Western Cape - Health,PPE000 Other PPEs,COMPASS MEDICAL WASTE SERVICES,0,28.59,44,0,31/12/2020,"R1,258.20",02/12/2021,"R1,258"
9,Western Cape - Health,PPE000 Other PPEs,COMPASS MEDICAL WASTE SERVICES,0,443.81,7,0,31/12/2020,"R3,106.67",17/02/2021,"R3,107"


In [ ]:
df=pd.DataFrame()
df_temp=pd.DataFrame()
i=0
for i in range(0,2800):
    df_previous_temp=df_temp.copy()
    for o in range(30):
        try:
            df_temp=get_values()
            break
        except Exception as err:
            if o<28:
                continue
            else:
                raise
    if df_temp.equals(df_previous_temp):
        i=i-1
        pass
    else:    
        df=df.append(df_temp)
        df.drop_duplicates(inplace=True)
        df.reset_index(inplace=True,drop=True)
        df.to_csv("data_005.csv")
        print(i)
    click_event()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  # This is added back by InteractiveShellApp.init_path()


0
1
2
3
4
5
6
7
8
9
10
11
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279


In [ ]:
driver.quit()